# calling libraries

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import os
import random
import tensorflow.keras as keras
# from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pywt

import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

In [ ]:
INPUT_SIZE = 4

In [ ]:
def find_intervals_without_missing_data(file_path):
    df = pd.read_csv(file_path)
    df['zaman_olcum'] = pd.to_datetime(df['zaman_olcum'], format='%d.%m.%Y %H:%M:%S')

    intervals = []
    start_idx = 0

    for i in range(1, len(df)):
        if (df.loc[i, 'zaman_olcum'] - df.loc[i-1, 'zaman_olcum']).seconds / 60 != 10:
            intervals.append((df.loc[start_idx, 'zaman_olcum'], df.loc[i-1, 'zaman_olcum'], i - start_idx))
            start_idx = i

    if start_idx < len(df):
        intervals.append((df.loc[start_idx, 'zaman_olcum'], df.loc[len(df)-1, 'zaman_olcum'], len(df) - start_idx))

    intervals.sort(key=lambda x: x[2], reverse=True)
    return intervals[0]

In [ ]:
def match_number_string(num1, num2):
    try:
        return float(num1) == float(num2)
    except ValueError:
        return False

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
# Define the directory containing the CSV files
raw_data_dir = 'StationDataRaw/'

def calculate_first_6_hours_arf(rainfall):
    """
    Calculate the Accumulated Rainfall (ARF).
    """
    ARF = 0
    for i in range(1, 37):
        ARF += rainfall[i]
 
    return ARF

def calculate_daily_total_rain(station_data_raw):
    station_data_raw['yagis_toplam_mm'] = station_data_raw['yagis_toplam_mm'].str.replace(',', '.')
    station_data_raw['yagis_toplam_mm'] = pd.to_numeric(station_data_raw['yagis_toplam_mm'])
    # Group by date and calculate the mean of the 'gunes_radyasyon' for each day
    daily_total_rain = station_data_raw.groupby('date')['yagis_toplam_mm'].sum().reset_index()

    return daily_total_rain

# Process each file and calculate the variables for each phenological stage
input_data_full = []
output_data_full = []
count = 0
for file_name in os.listdir(raw_data_dir):
    input_data = []
    output_data = []
    count += 1
    if file_name.endswith('.csv'):
        file_path = os.path.join(raw_data_dir, file_name)
        longest_interval = find_intervals_without_missing_data(file_path)

        if longest_interval[2] < 50000:
            continue
        # Extract the station code from the file name
        station_code = file_name.split('_')[0]
        raw_data = pd.read_csv(file_path, usecols=['zaman_olcum', 'sicaklik_2m', 'hava_basinci', 'nispi_nem_2m', 'yagis_toplam_mm'], dtype=str)
        # Convert 'zaman_olcum' to datetime
        raw_data['zaman_olcum'] = pd.to_datetime(raw_data['zaman_olcum'], format='%d.%m.%Y %H:%M:%S')

        filtered_raw_data = raw_data[(raw_data['zaman_olcum'] >= longest_interval[0]) & (raw_data['zaman_olcum'] <= longest_interval[1])]
        filtered_raw_data.reset_index(drop=True, inplace=True)

        time = filtered_raw_data['zaman_olcum']
        time.reset_index(drop=True, inplace=True)

        rf = filtered_raw_data['yagis_toplam_mm'].str.replace(',', '.')
        rf = pd.to_numeric(rf)
        rf.reset_index(drop=True, inplace=True)
        
        rh = filtered_raw_data['nispi_nem_2m'].str.replace(',', '.')
        rh = pd.to_numeric(rh)
        rh.reset_index(drop=True, inplace=True)
        
        temp = filtered_raw_data['sicaklik_2m'].str.replace(',', '.')
        temp = pd.to_numeric(temp)
        temp.reset_index(drop=True, inplace=True)

        pres = filtered_raw_data['hava_basinci'].str.replace(',', '.')
        pres = pd.to_numeric(pres)
        pres.reset_index(drop=True, inplace=True)
        
        print(count)
        
        if len(rf) < 31:
            continue

        isValid = True
        for i in range(24, len(rf)-5):
            arf_last_4_hours = []
            rf_last_4_data = []
            rh_last_4_data = []
            temp_last_4_data = []
            pres_last_4_data = []
            for j in range(4):
                if np.isnan(rf[i-4+j]) or np.isnan(rh[i-4+j]) or np.isnan(temp[i-4+j]) or np.isnan(pres[i-4+j]):
                    isValid = False
                    break
                one_hour_sum = 0
                for k in range(6):
                    if np.isnan(rf[(i-24)+6*j+k]):
                        isValid = False
                        break
                    one_hour_sum += rf[(i-24)+6*j+k]
                arf_last_4_hours.append(one_hour_sum)
                rf_last_4_data.append(rf[i-4+j])
                rh_last_4_data.append(rh[i-4+j])
                temp_last_4_data.append(temp[i-4+j])
                pres_last_4_data.append(pres[i-4+j])

            arf_next_1_hour = 0
            for j in range(6):
                if np.isnan(rf[i+j]):
                    isValid = False
                    break
                arf_next_1_hour += rf[i+j]

            raw_data_input = []
            raw_data_input.append(station_code)
            raw_data_input.append(time[i-1])
            raw_data_input.append(arf_last_4_hours)
            raw_data_input.append(rf_last_4_data)
            raw_data_input.append(rh_last_4_data)
            raw_data_input.append(temp_last_4_data)
            raw_data_input.append(pres_last_4_data)

            raw_data_output = []
            raw_data_output.append(station_code)
            raw_data_output.append(time[i-1])
            raw_data_output.append(arf_next_1_hour)

            input_data.append(raw_data_input)
            output_data.append(raw_data_output)

        if isValid:
            input_data_full += input_data
            output_data_full += output_data


In [ ]:
input_data_full = pd.DataFrame(input_data_full).rename({0: 'Station', 1: 'Date', 2: 'ARF', 3: 'RF', 4: 'RH', 5: 'T', 6: 'P'}, axis='columns')
output_data_full = pd.DataFrame(output_data_full).rename({0: 'Station', 1: 'Date', 2: 'ARF'}, axis='columns')

In [ ]:
pd.DataFrame(input_data_full).to_csv('input_dataset.csv', index=False)
pd.DataFrame(output_data_full).to_csv('output_dataset.csv', index=False)

In [ ]:
input_data =  pd.read_csv('input_dataset.csv')
output_data = pd.read_csv('output_dataset.csv')

In [ ]:
dataset = output_data

# Count the entries for each station where the 'ARF' column value is greater than 1e-5
station_arf_counts = dataset[dataset['ARF'] > 1e-5][dataset.columns[0]].value_counts()

# Convert the pandas Series to a list of tuples (station code, number of data entries)
station_arf_counts_list = list(station_arf_counts.items())

station_arf_counts_list = [elm for elm in station_arf_counts_list if elm[1] > 1500]

# Display the result
print("Station ARF counts as list of tuples:", station_arf_counts_list)
most_rain_recorded_stations = [pair[0] for pair in station_arf_counts_list]

# main functions

In [ ]:
#This is for getting the same results.
def reset_random_seeds(seed):
   os.environ['PYTHONHASHSEED']=str(seed)
   tf.random.set_seed(seed)
   np.random.seed(seed)
   random.seed(seed)
reset_random_seeds(42)

def successive(successive):
    input_data=[]
    for i in range(len(successive)-3):
        input_data.append([successive[i]]+[successive[i+1]]+[successive[i+2]]+[successive[i+3]])
        
    #return successive
    return input_data  

def preprocess_data(input_data, output_data):
    first_input_data = list(input_data['RF'])
    second_input_data = list(input_data['ARF'])
    third_input_data = list(input_data['RH'])
    fourth_input_data = list(input_data['T'])
    fifth_input_data = list(input_data['P'])
    output_data = list(output_data['ARF'])

    first_input_successive = first_input_data
    second_input_successive = second_input_data
    third_input_successive = third_input_data
    fourth_input_successive = fourth_input_data
    fifth_input_successive = fifth_input_data
    output_successive = output_data

    #division of data set into training and test data set
    N = len(first_input_data)
    division_of_training = 0.9
    first_input_train = first_input_successive[:int(N*division_of_training)]
    first_input_test = first_input_successive[int(N*division_of_training):int(N)]

    second_input_train = second_input_successive[:int(N*division_of_training)]
    second_input_test = second_input_successive[int(N*division_of_training):int(N)]

    third_input_train = third_input_successive[:int(N*division_of_training)]
    third_input_test = third_input_successive[int(N*division_of_training):int(N)]

    fourth_input_train = fourth_input_successive[:int(N*division_of_training)]
    fourth_input_test = fourth_input_successive[int(N*division_of_training):int(N)]

    fifth_input_train = fifth_input_successive[:int(N*division_of_training)]
    fifth_input_test = fifth_input_successive[int(N*division_of_training):int(N)]

    output_train = output_successive[:int(N*division_of_training)]
    output_test = output_successive[int(N*division_of_training):int(N)]

    return first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test

In [ ]:
#network configurations
hidden1=32
second_layer1=32
third_layer1=32
forth_layer1=16

hidden2=32
second_layer2=16
third_layer2=16
forth_layer2=8

hidden3=32
second_layer3=16
third_layer3=16
forth_layer3=8

hidden4=32
second_layer4=16
third_layer4=16
forth_layer4=8

hidden5=32
second_layer5=16
third_layer5=16
forth_layer5=8

hidden6=32
second_layer6=16
third_layer6=16
forth_layer6=8

In [ ]:
def PECNET2(first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test):
    
    def standardize_and_normalize(data, epsilon=1e-8):
        mean = np.mean(data, axis=1, keepdims=True)
        std = np.std(data, axis=1, keepdims=True)
        standardized_data = (data - mean) / (std + epsilon)
        return standardized_data

    # Standardize and normalize the data
    first_input_train = standardize_and_normalize(first_input_train)
    first_input_test = standardize_and_normalize(first_input_test)
    
    second_input_train = standardize_and_normalize(second_input_train)
    second_input_test = standardize_and_normalize(second_input_test)
    
    third_input_train = standardize_and_normalize(third_input_train)
    third_input_test = standardize_and_normalize(third_input_test)
    
    fourth_input_train = standardize_and_normalize(fourth_input_train)
    fourth_input_test = standardize_and_normalize(fourth_input_test)

    fifth_input_train = standardize_and_normalize(fifth_input_train)
    fifth_input_test = standardize_and_normalize(fifth_input_test)
    
    X_train=np.array(first_input_train)
    y_train=np.array(output_train)

    X_test=np.array(first_input_test)
    y_test=np.array(output_test)

    m_primary=len(X_train[0,:])
    p_primary=np.size(y_train[0])
    N_primary=len(X_train)

    model= Sequential ([
        Dense(hidden1, input_dim=m_primary, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_primary)
        ])
        
    #model.summary()
    # Define a learning rate scheduler
    def scheduler(epoch, lr):
        if epoch < 100:
            return lr
        elif epoch < 200:
            return lr * 0.5
        else:
            return lr * 0.2

    lr_scheduler = LearningRateScheduler(scheduler)

    # Define early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=50, restore_best_weights=True)

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.1,momentum=0.75, decay=0.0)
    adam = keras.optimizers.legacy.Adam(learning_rate=0.0005)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse','mae','mape'])

    # Training the model with learning rate scheduler and early stopping
    history1=model.fit(X_train, y_train, batch_size=128, epochs=300, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])  

    predicted_train = model.predict(X_train) 
    predicted_train = np.reshape(predicted_train, (predicted_train.size,))
    error_train1=predicted_train-y_train

    predicted_test = model.predict(X_test) 
    predicted_test = np.reshape(predicted_test, (predicted_test.size,))
    error_test1=predicted_test-y_test
    
    ######## Second NN

    error_train1a=pd.DataFrame(error_train1)
    add_train1=second_input_train
    
    X_error_train1=np.array(add_train1)
    y_error_train1=np.array(error_train1a)

    error_test1a=pd.DataFrame(error_test1)
    add_test1=second_input_test

    X_error_test1=np.array(add_test1)

    m_second=len(X_error_train1[0,:])
    p_second=np.size(y_train[0])
    N_second=len(X_error_train1)

    error_model1= Sequential ([
        Dense(hidden2, input_dim=m_second, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_second)
    ])

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    error_model1.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history2=error_model1.fit(X_error_train1, y_error_train1, batch_size=256, epochs=300, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])

    error_predicted_tr1 = error_model1.predict(X_error_train1)
    error_predicted_tr1 = np.reshape(error_predicted_tr1, (error_predicted_tr1.size,))
    error_predicted_tes1 = error_model1.predict(X_error_test1)
    error_predicted_tes1 = np.reshape(error_predicted_tes1, (error_predicted_tes1.size,))

    error_train2=(error_train1-error_predicted_tr1)
    error_test2=(error_test1-error_predicted_tes1)

    ########## Third NN

    error_train2a=pd.DataFrame(error_train2)
    add_train2=third_input_train 
    
    X_error_train2=np.array(add_train2)
    y_error_train2=np.array(error_train2a)

    error_test2a=pd.DataFrame(error_test2)
    add_test2=third_input_test

    X_error_test2=np.array(add_test2)

    m_third=len(X_error_train2[0,:])
    p_third=np.size(y_error_train2[0])
    N_third=len(X_error_train2)

    error_model2= Sequential ([
        Dense(hidden3, input_dim=m_third, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_third)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model2.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.00005), metrics=['mse','mae','mape'])
    history3=error_model2.fit(X_error_train2, y_error_train2, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr2 = error_model2.predict(X_error_train2)
    error_predicted_tr2 = np.reshape(error_predicted_tr2, (error_predicted_tr2.size,))
    error_predicted_tes2 = error_model2.predict(X_error_test2)
    error_predicted_tes2 = np.reshape(error_predicted_tes2, (error_predicted_tes2.size,))

    error_train3 = (error_train2-error_predicted_tr2)
    error_test3 = (error_test2-error_predicted_tes2)

    ########## Fourth NN

    error_train3a=pd.DataFrame(error_train3)
    add_train3=fourth_input_train
    
    X_error_train3=np.array(add_train3)
    y_error_train3=np.array(error_train3a)

    error_test3a=pd.DataFrame(error_test3)
    add_test3=fourth_input_test

    X_error_test3=np.array(add_test3)

    m_fourth=len(X_error_train3[0,:])
    p_fourth=np.size(y_error_train3[0])
    N_fourth=len(X_error_train3)

    error_model3= Sequential ([
        Dense(hidden4, input_dim=m_fourth, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_fourth)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model3.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.00005), metrics=['mse','mae','mape'])
    history4=error_model3.fit(X_error_train3, y_error_train3, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr3 = error_model3.predict(X_error_train3)
    error_predicted_tr3 = np.reshape(error_predicted_tr3, (error_predicted_tr3.size,))
    error_predicted_tes3 = error_model3.predict(X_error_test3)
    error_predicted_tes3 = np.reshape(error_predicted_tes3, (error_predicted_tes3.size,))

    error_train4 = (error_train3-error_predicted_tr3)
    error_test4 = (error_test3-error_predicted_tes3)

    ########## Fifth NN

    error_train4a=pd.DataFrame(error_train4)
    add_train4=fifth_input_train
    
    X_error_train4=np.array(add_train4)
    y_error_train4=np.array(error_train4a)

    error_test4a=pd.DataFrame(error_test4)
    add_test4=fifth_input_test

    X_error_test4=np.array(add_test4)

    m_fifth=len(X_error_train4[0,:])
    p_fifth=np.size(y_error_train4[0])
    N_fifth=len(X_error_train4)

    error_model4= Sequential ([
        Dense(hidden5, input_dim=m_fifth, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_fifth)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model4.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.00005), metrics=['mse','mae','mape'])
    history5=error_model4.fit(X_error_train4, y_error_train4, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr4 = error_model4.predict(X_error_train4)
    error_predicted_tr4 = np.reshape(error_predicted_tr4, (error_predicted_tr4.size,))
    error_predicted_tes4 = error_model4.predict(X_error_test4)
    error_predicted_tes4 = np.reshape(error_predicted_tes4, (error_predicted_tes4.size,))

    error_train5 = (error_train4-error_predicted_tr4)
    error_test5 = (error_test4-error_predicted_tes4)

    ##### Sixth NN (Error Network)

    error_train5a=pd.DataFrame(error_train5)
    for i in range(INPUT_SIZE):
        error_train5a[i+1]= error_train5a[i].shift(1)
    error_train5a = error_train5a.replace(np.nan, 0)
    error_train5a = error_train5a.iloc[:,::-1]
    ##error normalization
    subtraction_error_train5=np.mean(np.array(error_train5a)[:,:-1], axis=1)
    error_train5a -= subtraction_error_train5[:, None]
    error_train5a = np.array(error_train5a)
    days_train = error_train5a[:,:INPUT_SIZE]
    input5_train=days_train
    output5_train=error_train5a[:,INPUT_SIZE]

    X_error_train5=np.array(input5_train)
    y_error_train5=np.array(output5_train)

    error_test5a=pd.DataFrame(error_test5)
    for i in range(INPUT_SIZE):
        error_test5a[i+1]= error_test5a[i].shift(1)
    error_test5a = error_test5a.replace(np.nan, 0)
    error_test5a = error_test5a.iloc[:,::-1]

    subtraction_error_test5=np.array(error_test5a)
    subtraction_error_test5=subtraction_error_test5[:,:-1]
    subtraction_error_test5=np.mean(subtraction_error_test5, axis=1)
    error_test5a -= subtraction_error_test5[:, None]
    error_test5a=np.array(error_test5a)
    days_test = error_test5a[:,:INPUT_SIZE]
    input5_test=days_test
    output5_test=error_test5a[:,INPUT_SIZE]

    X_error_test5=np.array(input5_test)
    
    m_error=len(X_error_train5[0,:])
    p_error=np.size(y_error_train5[0])
    N_error=len(X_error_train5)

    error_model5= Sequential ([
        Dense(hidden5, input_dim=m_error, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_error)
    ])

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    error_model5.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history6=error_model5.fit(X_error_train5, y_error_train5, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])

    error_predicted_tr5 = error_model5.predict(X_error_train5)
    error_predicted_tr5 = np.reshape(error_predicted_tr5, (error_predicted_tr5.size,))
    error_predicted_tes5 = error_model5.predict(X_error_test5)
    error_predicted_tes5= np.reshape(error_predicted_tes5, (error_predicted_tes5.size,))

    compensated_y_train=error_train5-(error_predicted_tr5+subtraction_error_train5)
    compensated_y_test=error_test5-(error_predicted_tes5+subtraction_error_test5)

    error_predicted_tr6=error_predicted_tr5+subtraction_error_train5
    error_predicted_tes6=error_predicted_tes5+subtraction_error_test5

    training_final_add=np.column_stack((predicted_train, error_predicted_tr1))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr2))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr3))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr4))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr6))

    test_final_add=np.column_stack((predicted_test, error_predicted_tes1))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes2))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes3))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes4))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes6))

    ####final NN
    m_final=len(training_final_add[0,:])
    p_final=np.size(y_train[0])
    N_final=len(training_final_add)

    final_model= Sequential ([
        Dense(hidden6, input_dim=m_final, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_final)
    ])

    #final_model.summary()

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    final_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse','mae','mape'])
    final_history=final_model.fit(training_final_add, y_train, batch_size=256, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])

    final_predicted_tr = final_model.predict(training_final_add)
    final_predicted_tr = np.reshape(final_predicted_tr, (final_predicted_tr.size,))
    final_predicted_tes = final_model.predict(test_final_add)
    final_predicted_tes = np.reshape(final_predicted_tes, (final_predicted_tes.size,))
    
    return final_predicted_tes, history1, history2, history3, history4, history5, history6, final_history

## PECNET function

In [ ]:
def PECNET(first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test):
    
    def standardize_and_normalize(data, epsilon=1e-8):
        mean = np.mean(data, axis=1, keepdims=True)
        std = np.std(data, axis=1, keepdims=True)
        standardized_data = (data - mean) / (std + epsilon)
        return standardized_data

    # Standardize and normalize the data
    first_input_train = standardize_and_normalize(first_input_train)
    first_input_test = standardize_and_normalize(first_input_test)
    
    second_input_train = standardize_and_normalize(second_input_train)
    second_input_test = standardize_and_normalize(second_input_test)
    
    third_input_train = standardize_and_normalize(third_input_train)
    third_input_test = standardize_and_normalize(third_input_test)
    
    fourth_input_train = standardize_and_normalize(fourth_input_train)
    fourth_input_test = standardize_and_normalize(fourth_input_test)

    fifth_input_train = standardize_and_normalize(fifth_input_train)
    fifth_input_test = standardize_and_normalize(fifth_input_test)
    
    X_train=np.array(first_input_train)
    y_train=np.array(output_train)

    X_test=np.array(first_input_test)
    y_test=np.array(output_test)

    m_primary=len(X_train[0,:])
    p_primary=np.size(y_train[0])
    N_primary=len(X_train)

    model= Sequential ([
        Dense(hidden1, input_dim=m_primary, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.4),  # Increased dropout rate
        Dense(second_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.4),  # Increased dropout rate
        Dense(third_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.4),  # Increased dropout rate
        Dense(forth_layer1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.4),  # Increased dropout rate
        Dense(p_primary)
        ])
        
    #model.summary()
    # Define a learning rate scheduler
    def scheduler(epoch, lr):
        if epoch < 50:
            return lr
        elif epoch < 100:
            return lr * 0.5
        elif epoch < 150:
            return lr * 0.5
        else:
            return lr * 0.2

    lr_scheduler = LearningRateScheduler(scheduler)

    # Define early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=50, restore_best_weights=True)

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.1,momentum=0.75, decay=0.0)
    adam = keras.optimizers.legacy.Adam(learning_rate=0.00025)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse','mae','mape'])

    # Training the model with learning rate scheduler and early stopping
    history1=model.fit(X_train, y_train, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])  

    predicted_train = model.predict(X_train) 
    predicted_train = np.reshape(predicted_train, (predicted_train.size,))
    error_train1=predicted_train-y_train

    predicted_test = model.predict(X_test) 
    predicted_test = np.reshape(predicted_test, (predicted_test.size,))
    error_test1=predicted_test-y_test
    
    ######## Second NN

    error_train1a=pd.DataFrame(error_train1)
    add_train1=second_input_train
    
    X_error_train1=np.array(add_train1)
    y_error_train1=np.array(error_train1a)

    error_test1a=pd.DataFrame(error_test1)
    add_test1=second_input_test

    X_error_test1=np.array(add_test1)

    m_second=len(X_error_train1[0,:])
    p_second=np.size(y_train[0])
    N_second=len(X_error_train1)

    error_model1= Sequential ([
        Dense(hidden2, input_dim=m_second, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.4),
        Dense(second_layer1), #,activation='relu'),
        Dropout(0.4),
        Dense(third_layer1), #,activation='relu'),
        Dropout(0.4),
        Dense(forth_layer1), #,activation='relu'),
        Dropout(0.4),
        Dense(p_second)
    ])

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    error_model1.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.00025), metrics=['mse','mae','mape'])
    history2=error_model1.fit(X_error_train1, y_error_train1, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])

    error_predicted_tr1 = error_model1.predict(X_error_train1)
    error_predicted_tr1 = np.reshape(error_predicted_tr1, (error_predicted_tr1.size,))
    error_predicted_tes1 = error_model1.predict(X_error_test1)
    error_predicted_tes1 = np.reshape(error_predicted_tes1, (error_predicted_tes1.size,))

    error_train2=(error_train1-error_predicted_tr1)
    error_test2=(error_test1-error_predicted_tes1)

    ########## Third NN

    error_train2a=pd.DataFrame(error_train2)
    add_train2=third_input_train 
    
    X_error_train2=np.array(add_train2)
    y_error_train2=np.array(error_train2a)

    error_test2a=pd.DataFrame(error_test2)
    add_test2=third_input_test

    X_error_test2=np.array(add_test2)

    m_third=len(X_error_train2[0,:])
    p_third=np.size(y_error_train2[0])
    N_third=len(X_error_train2)

    error_model2= Sequential ([
        Dense(hidden3, input_dim=m_third, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.2),
        Dense(second_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(third_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(forth_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(p_third)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model2.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history3=error_model2.fit(X_error_train2, y_error_train2, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr2 = error_model2.predict(X_error_train2)
    error_predicted_tr2 = np.reshape(error_predicted_tr2, (error_predicted_tr2.size,))
    error_predicted_tes2 = error_model2.predict(X_error_test2)
    error_predicted_tes2 = np.reshape(error_predicted_tes2, (error_predicted_tes2.size,))

    error_train3 = (error_train2-error_predicted_tr2)
    error_test3 = (error_test2-error_predicted_tes2)

    ########## Fourth NN

    error_train3a=pd.DataFrame(error_train3)
    add_train3=fourth_input_train
    
    X_error_train3=np.array(add_train3)
    y_error_train3=np.array(error_train3a)

    error_test3a=pd.DataFrame(error_test3)
    add_test3=fourth_input_test

    X_error_test3=np.array(add_test3)

    m_fourth=len(X_error_train3[0,:])
    p_fourth=np.size(y_error_train3[0])
    N_fourth=len(X_error_train3)

    error_model3= Sequential ([
        Dense(hidden4, input_dim=m_fourth, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.2),
        Dense(second_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(third_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(forth_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(p_fourth)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model3.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history4=error_model3.fit(X_error_train3, y_error_train3, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr3 = error_model3.predict(X_error_train3)
    error_predicted_tr3 = np.reshape(error_predicted_tr3, (error_predicted_tr3.size,))
    error_predicted_tes3 = error_model3.predict(X_error_test3)
    error_predicted_tes3 = np.reshape(error_predicted_tes3, (error_predicted_tes3.size,))

    error_train4 = (error_train3-error_predicted_tr3)
    error_test4 = (error_test3-error_predicted_tes3)

    ########## Fifth NN

    error_train4a=pd.DataFrame(error_train4)
    add_train4=fifth_input_train
    
    X_error_train4=np.array(add_train4)
    y_error_train4=np.array(error_train4a)

    error_test4a=pd.DataFrame(error_test4)
    add_test4=fifth_input_test

    X_error_test4=np.array(add_test4)

    m_fifth=len(X_error_train4[0,:])
    p_fifth=np.size(y_error_train4[0])
    N_fifth=len(X_error_train4)

    error_model4= Sequential ([
        Dense(hidden5, input_dim=m_fifth, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.2),
        Dense(second_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(third_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(forth_layer2), #,activation='relu'),
        Dropout(0.2),
        Dense(p_fifth)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.1,momentum=0.75, decay=0.0, nesterov=False)
    error_model4.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history5=error_model4.fit(X_error_train4, y_error_train4, batch_size=512, epochs=300, shuffle=False, verbose=0, callbacks=[early_stopping])  

    error_predicted_tr4 = error_model4.predict(X_error_train4)
    error_predicted_tr4 = np.reshape(error_predicted_tr4, (error_predicted_tr4.size,))
    error_predicted_tes4 = error_model4.predict(X_error_test4)
    error_predicted_tes4 = np.reshape(error_predicted_tes4, (error_predicted_tes4.size,))

    error_train5 = (error_train4-error_predicted_tr4)
    error_test5 = (error_test4-error_predicted_tes4)

    ##### Sixth NN (Error Network)

    error_train5a=pd.DataFrame(error_train5)
    for i in range(INPUT_SIZE):
        error_train5a[i+1]= error_train5a[i].shift(1)
    error_train5a = error_train5a.replace(np.nan, 0)
    error_train5a = error_train5a.iloc[:,::-1]
    ##error normalization
    subtraction_error_train5=np.mean(np.array(error_train5a)[:,:-1], axis=1)
    error_train5a -= subtraction_error_train5[:, None]
    error_train5a = np.array(error_train5a)
    days_train = error_train5a[:,:INPUT_SIZE]
    input5_train=days_train
    output5_train=error_train5a[:,INPUT_SIZE]

    X_error_train5=np.array(input5_train)
    y_error_train5=np.array(output5_train)

    error_test5a=pd.DataFrame(error_test5)
    for i in range(INPUT_SIZE):
        error_test5a[i+1]= error_test5a[i].shift(1)
    error_test5a = error_test5a.replace(np.nan, 0)
    error_test5a = error_test5a.iloc[:,::-1]

    subtraction_error_test5=np.array(error_test5a)
    subtraction_error_test5=subtraction_error_test5[:,:-1]
    subtraction_error_test5=np.mean(subtraction_error_test5, axis=1)
    error_test5a -= subtraction_error_test5[:, None]
    error_test5a=np.array(error_test5a)
    days_test = error_test5a[:,:INPUT_SIZE]
    input5_test=days_test
    output5_test=error_test5a[:,INPUT_SIZE]

    X_error_test5=np.array(input5_test)
    
    m_error=len(X_error_train5[0,:])
    p_error=np.size(y_error_train5[0])
    N_error=len(X_error_train5)

    error_model5= Sequential ([
        Dense(hidden5, input_dim=m_error, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.2),  # Increased dropout rate
        Dense(second_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(third_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(forth_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(p_error)
    ])

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    error_model5.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    history6=error_model5.fit(X_error_train5, y_error_train5, batch_size=512, epochs=500, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])

    error_predicted_tr5 = error_model5.predict(X_error_train5)
    error_predicted_tr5 = np.reshape(error_predicted_tr5, (error_predicted_tr5.size,))
    error_predicted_tes5 = error_model5.predict(X_error_test5)
    error_predicted_tes5= np.reshape(error_predicted_tes5, (error_predicted_tes5.size,))

    compensated_y_train=error_train5-(error_predicted_tr5+subtraction_error_train5)
    compensated_y_test=error_test5-(error_predicted_tes5+subtraction_error_test5)

    error_predicted_tr6=error_predicted_tr5+subtraction_error_train5
    error_predicted_tes6=error_predicted_tes5+subtraction_error_test5

    training_final_add=np.column_stack((predicted_train, error_predicted_tr1))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr2))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr3))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr4))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr6))

    test_final_add=np.column_stack((predicted_test, error_predicted_tes1))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes2))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes3))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes4))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes6))

    ####final NN
    m_final=len(training_final_add[0,:])
    p_final=np.size(y_train[0])
    N_final=len(training_final_add)

    final_model= Sequential ([
        Dense(hidden6, input_dim=m_final, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        Dropout(0.2),  # Increased dropout rate
        Dense(second_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(third_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(forth_layer1),
        Dropout(0.2),  # Increased dropout rate
        Dense(p_final)
    ])

    #final_model.summary()

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    final_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0005), metrics=['mse','mae','mape'])
    final_history=final_model.fit(training_final_add, y_train, batch_size=512, epochs=500, shuffle=False, verbose=0, callbacks=[lr_scheduler, early_stopping])

    final_predicted_tr = final_model.predict(training_final_add)
    final_predicted_tr = np.reshape(final_predicted_tr, (final_predicted_tr.size,))
    final_predicted_tes = final_model.predict(test_final_add)
    final_predicted_tes = np.reshape(final_predicted_tes, (final_predicted_tes.size,))
    
    return final_predicted_tes, y_test, history1, history2, history3, history4, history5, history6, final_history

## LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from sklearn.metrics import mean_absolute_percentage_error
import ast

def build_lstm_model(input_data, output_data):
    # Ensure 'Date' columns are of datetime type
    input_data['Date'] = pd.to_datetime(input_data['Date'])
    output_data['Date'] = pd.to_datetime(output_data['Date'])

    # Merge datasets on 'Station' and 'Date'
    data = pd.merge(input_data, output_data, on=['Station', 'Date'], suffixes=('_input', ''))

    def restructure_data(data):
        # Stacking the columns to form single columns for each feature
        df_list = []
        for i in range(4):  # for RF_0, RF_1, RF_2, RF_3
            temp_df = data[['Station', 'Date']]
            temp_df['RF'] = data[f'RF_{i}']
            temp_df['ARF_input'] = data[f'ARF_{i}']
            temp_df['RH'] = data[f'RH_{i}']
            temp_df['T'] = data[f'T_{i}']
            temp_df['P'] = data[f'P_{i}']
            temp_df['ARF'] = data['ARF']
            df_list.append(temp_df)

        # Concatenate all the dataframes
        stacked_df = pd.concat(df_list, axis=0)

        # Sort the dataframe by 'Station' and 'Date'
        stacked_df = stacked_df.sort_values(by=['Station', 'Date']).reset_index(drop=True)

        return stacked_df
    
    # Function to create input-output pairs
    def create_input_output_pairs(data, sequence_length=4):
        X, y = [], []
        feature_columns = [col for col in data.columns if col not in ['Date', 'Station', 'ARF']]
        for i in range(0, len(data), 4):
            X.append(data.iloc[i:i+sequence_length][feature_columns].values)
            y.append(data.iloc[i]['ARF'])
        return np.array(X), np.array(y)
    
    stacked_data = restructure_data(data)
    X, y = create_input_output_pairs(stacked_data)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=False)
    
    # Standardize the data
    scaler = StandardScaler()
    X_train_shape = X_train.shape
    X_test_shape = X_test.shape
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train_shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test_shape)

    # Build LSTM model
    model_lstm = Sequential()
    model_lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1))
    model_lstm.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    history_lstm = model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Predict using the model
    y_pred_lstm = model_lstm.predict(X_test)

    return y_pred_lstm, y_test, history_lstm


## Prophet

In [ ]:
from prophet import Prophet

def prophet(input_data_next, output_data_next):
    # Convert 'Date' to datetime format
    input_data_next['Date'] = pd.to_datetime(input_data_next['Date'])
    output_data_next['Date'] = pd.to_datetime(output_data_next['Date'])
    output_data_next = output_data_next.rename(columns={'ARF': 'y'})
    
    # Merge the shifted input data with the output data
    prophet_data = pd.merge(input_data_next, output_data_next, on=['Station', 'Date'], suffixes=('_input', ''))
    prophet_data = prophet_data.rename(columns={'Date': 'ds'})
    prophet_data = prophet_data.dropna().reset_index(drop=True)  # Drop rows with NaN values resulting from the shift

    # Initialize and fit Prophet model
    prophet_model = Prophet()
    
    # Add regressors for each additional feature
    additional_features = [col for col in input_data_next.columns if col not in ['Date', 'Station']]
    for feature in additional_features:
        prophet_model.add_regressor(feature)

    # Fit the model
    prophet_model.fit(prophet_data[['ds', 'y'] + additional_features])
    
    # Prepare future dataframe for predictions
    future_dates = prophet_data[['ds']].tail(len(output_data_next)).reset_index(drop=True)
    future_features = input_data_next[additional_features].tail(len(output_data_next)).reset_index(drop=True)
    future_dates = future_dates.join(future_features)

    # Make predictions
    prophet_forecast = prophet_model.predict(future_dates)
    prophet_predictions = prophet_forecast['yhat'].values
    prophet_output_test = np.array(output_data_next['y'])
    length = int(len(prophet_output_test) * 0.9)
    return prophet_predictions[length:], prophet_output_test[length:]


In [ ]:
def run_pecnet(input_data_next, output_data_next, pecnet_preds, station_id):
    pecnet_preds[station_id] = []
    first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test = preprocess_data(input_data_next, output_data_next)
    # Predict
    print(len(output_test))
    pecnet_predictions, pecnet_test, history1, history2, history3, history4, history5, history6, final_history = PECNET(first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test)

    pecnet_predictions = np.array(pecnet_predictions)
    pecnet_test = np.array(pecnet_test)

    pecnet_preds[station_id].append(pecnet_predictions)
    pecnet_preds[station_id].append(pecnet_test)
    pecnet_preds[station_id].append([history1, history2, history3, history4, history5, history6, final_history])
    pecnet_preds[station_id].append(mean_absolute_percentage_error(pecnet_test, pecnet_predictions))
    pecnet_preds[station_id].append(mean_squared_error(pecnet_test, pecnet_predictions))
    pecnet_preds[station_id].append(mean_absolute_error(pecnet_test, pecnet_predictions))

    # Filter out target values less than 1e-5
    mask1 = pecnet_test >= 1e-5
    output_test_filtered = pecnet_test[mask1]
    pecnet_predictions_filtered = pecnet_predictions[mask1]

    pecnet_preds[station_id].append(mean_absolute_percentage_error(output_test_filtered, pecnet_predictions_filtered))
    pecnet_preds[station_id].append(mean_squared_error(output_test_filtered, pecnet_predictions_filtered))
    pecnet_preds[station_id].append(mean_absolute_error(output_test_filtered, pecnet_predictions_filtered))

    return pecnet_preds

## Preprocessing and predictions

In [ ]:
def run_multiple(input_data_next, output_data_next, pecnet_preds, lstm_preds, prophet_preds, station_id):
    pecnet_preds[station_id] = []
    lstm_preds[station_id] = []
    prophet_preds[station_id] = []
    first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test = preprocess_data(input_data_next, output_data_next)
    
    in_data = input_data_next.copy()

    df = pd.DataFrame(list(in_data['RF']), index=in_data.index)
    df.reset_index(drop=True, inplace=True)
    df.columns = [f"{'RF'}_{i}" for i in range(df.shape[1])]
    df_temp = pd.DataFrame(list(in_data['ARF']), index=in_data.index)
    df_temp.columns = [f"{'ARF'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['RH']), index=in_data.index)
    df_temp.columns = [f"{'RH'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['T']), index=in_data.index)
    df_temp.columns = [f"{'T'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['P']), index=in_data.index)
    df_temp.columns = [f"{'P'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['Date']), index=in_data.index)
    df_temp.columns = [f"{'Date'}"]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df_temp, df], axis=1)
    df_temp = pd.DataFrame(list(in_data['Station']), index=in_data.index)
    df_temp.columns = [f"{'Station'}"]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df_temp, df], axis=1)

    in_data_next = df
    in_data_next.reset_index(drop=True, inplace=True)
    in_data_next = in_data_next.fillna(method='ffill').fillna(method='bfill')

    # Predict
    pecnet_predictions, pecnet_test, history1, history2, history3, history4, history5, history6, final_history = PECNET(first_input_train, first_input_test, second_input_train, second_input_test, third_input_train, third_input_test, fourth_input_train, fourth_input_test, fifth_input_train, fifth_input_test, output_train, output_test)
    lstm_predictions, lstm_test, history_lstm = build_lstm_model(in_data_next, output_data_next)
    prophet_predictions, prophet_test = prophet(in_data_next, output_data_next)

    pecnet_predictions = np.array(pecnet_predictions)
    lstm_predictions = np.array(lstm_predictions)
    prophet_predictions = np.array(prophet_predictions)

    pecnet_test = np.array(pecnet_test)
    lstm_test = np.array(lstm_test)
    prophet_test = np.array(prophet_test)

    pecnet_preds[station_id].append(pecnet_predictions)
    pecnet_preds[station_id].append(pecnet_test)
    pecnet_preds[station_id].append([history1, history2, history3, history4, history5, history6, final_history])
    lstm_preds[station_id].append(lstm_predictions)
    lstm_preds[station_id].append(lstm_test)
    lstm_preds[station_id].append(history_lstm)
    prophet_preds[station_id].append(prophet_predictions)
    prophet_preds[station_id].append(prophet_test)
    prophet_preds[station_id].append(None)
    pecnet_preds[station_id].append(mean_absolute_percentage_error(pecnet_test, pecnet_predictions))
    pecnet_preds[station_id].append(mean_squared_error(pecnet_test, pecnet_predictions))
    pecnet_preds[station_id].append(mean_absolute_error(pecnet_test, pecnet_predictions))
    lstm_preds[station_id].append(mean_absolute_percentage_error(lstm_test, lstm_predictions))
    lstm_preds[station_id].append(mean_squared_error(lstm_test, lstm_predictions))
    lstm_preds[station_id].append(mean_absolute_error(lstm_test, lstm_predictions))
    prophet_preds[station_id].append(mean_absolute_percentage_error(prophet_test, prophet_predictions))
    prophet_preds[station_id].append(mean_squared_error(prophet_test, prophet_predictions))
    prophet_preds[station_id].append(mean_absolute_error(prophet_test, prophet_predictions))

    # Filter out target values less than 1e-5
    mask1 = pecnet_test >= 1e-5
    pecnet_test_filtered = pecnet_test[mask1]
    pecnet_predictions_filtered = pecnet_predictions[mask1]
    # Filter out target values less than 1e-5
    mask2 = lstm_test >= 1e-5
    lstm_test_filtered = lstm_test[mask2]
    lstm_predictions_filtered = lstm_predictions[mask2]
    # Filter out target values less than 1e-5
    mask3 = prophet_test >= 1e-5
    prophet_test_filtered = prophet_test[mask3]
    prophet_predictions_filtered = prophet_predictions[mask3]

    pecnet_preds[station_id].append(mean_absolute_percentage_error(pecnet_test_filtered, pecnet_predictions_filtered))
    pecnet_preds[station_id].append(mean_squared_error(pecnet_test_filtered, pecnet_predictions_filtered))
    pecnet_preds[station_id].append(mean_absolute_error(pecnet_test_filtered, pecnet_predictions_filtered))
    lstm_preds[station_id].append(mean_absolute_percentage_error(lstm_test_filtered, lstm_predictions_filtered))
    lstm_preds[station_id].append(mean_squared_error(lstm_test_filtered, lstm_predictions_filtered))
    lstm_preds[station_id].append(mean_absolute_error(lstm_test_filtered, lstm_predictions_filtered))
    prophet_preds[station_id].append(mean_absolute_percentage_error(prophet_test_filtered, prophet_predictions_filtered))
    prophet_preds[station_id].append(mean_squared_error(prophet_test_filtered, prophet_predictions_filtered))
    prophet_preds[station_id].append(mean_absolute_error(prophet_test_filtered, prophet_predictions_filtered))

    return pecnet_preds, lstm_preds, prophet_preds

In [ ]:
lstm_preds = {}
for i in range(len(most_rain_recorded_stations)): 
    input_data_next = input_data[input_data[input_data.columns[0]] == most_rain_recorded_stations[i]]
    output_data_next = output_data[output_data[output_data.columns[0]] == most_rain_recorded_stations[i]]

    in_data = input_data_next.copy()

    df = pd.DataFrame(list(in_data['RF']), index=in_data.index)
    df.reset_index(drop=True, inplace=True)
    df.columns = [f"{'RF'}_{i}" for i in range(df.shape[1])]
    df_temp = pd.DataFrame(list(in_data['ARF']), index=in_data.index)
    df_temp.columns = [f"{'ARF'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['RH']), index=in_data.index)
    df_temp.columns = [f"{'RH'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['T']), index=in_data.index)
    df_temp.columns = [f"{'T'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['P']), index=in_data.index)
    df_temp.columns = [f"{'P'}_{i}" for i in range(df_temp.shape[1])]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_temp], axis=1)
    df_temp = pd.DataFrame(list(in_data['Date']), index=in_data.index)
    df_temp.columns = [f"{'Date'}"]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df_temp, df], axis=1)
    df_temp = pd.DataFrame(list(in_data['Station']), index=in_data.index)
    df_temp.columns = [f"{'Station'}"]
    df_temp.reset_index(drop=True, inplace=True)
    df = pd.concat([df_temp, df], axis=1)

    in_data_next = df
    in_data_next.reset_index(drop=True, inplace=True)
    in_data_next = in_data_next.fillna(method='ffill').fillna(method='bfill')

    y_pred_lstm, y_test, history_lstm = build_lstm_model(in_data_next, output_data_next)

    station_id = most_rain_recorded_stations[i]
    lstm_preds[station_id] = []
    lstm_preds[station_id].append(y_pred_lstm)
    lstm_preds[station_id].append(y_test)
    lstm_preds[station_id].append(history_lstm)
    lstm_preds[station_id].append(mean_absolute_percentage_error(y_test, y_pred_lstm))
    lstm_preds[station_id].append(mean_squared_error(y_test, y_pred_lstm))
    lstm_preds[station_id].append(mean_absolute_error(y_test, y_pred_lstm))
    mask2 = y_test >= 1e-5
    lstm_test_filtered = y_test[mask2]
    lstm_predictions_filtered = y_pred_lstm[mask2]
    lstm_preds[station_id].append(mean_absolute_percentage_error(lstm_test_filtered, lstm_predictions_filtered))
    lstm_preds[station_id].append(mean_squared_error(lstm_test_filtered, lstm_predictions_filtered))
    lstm_preds[station_id].append(mean_absolute_error(lstm_test_filtered, lstm_predictions_filtered))
    #prophet_predictions, prophet_output_test = prophet(in_data_next, output_data_next)

In [ ]:
most_rain_recorded_stations = [pair[0] for pair in station_arf_counts_list]
#anomalies = [most_rain_recorded_stations[2], most_rain_recorded_stations[11]]
#most_rain_recorded_stations = most_rain_recorded_stations[:1]
pecnet_preds = {}
lstm_preds = {}
prophet_preds = {}

#most_rain_recorded_stations.pop(1)
input_data = input_data.fillna(method='ffill').fillna(method='bfill')
output_data = output_data.fillna(method='ffill').fillna(method='bfill')

for i in range(len(most_rain_recorded_stations)):
    input_data_next = input_data[input_data[input_data.columns[0]] == most_rain_recorded_stations[i]]
    output_data_next = output_data[output_data[output_data.columns[0]] == most_rain_recorded_stations[i]]
    pecnet_preds, lstm_preds, prophet_preds = run_multiple(input_data_next, output_data_next, pecnet_preds, lstm_preds, prophet_preds, most_rain_recorded_stations[i])
    #pecnet_preds = run_pecnet(input_data_next, output_data_next, pecnet_preds, most_rain_recorded_stations[i])


In [ ]:
mask9 = y_test > 1e-5
mean_absolute_percentage_error(y_test[mask9], y_pred_lstm[mask9])

In [ ]:
most_rain_recorded_stations = [pair[0] for pair in station_arf_counts_list]
#anomalies = [most_rain_recorded_stations[2], most_rain_recorded_stations[11]]
#most_rain_recorded_stations = most_rain_recorded_stations[:1]
pecnet_preds = {}
lstm_preds = {}
prophet_preds = {}

#most_rain_recorded_stations.pop(1)
input_data = input_data.fillna(method='ffill').fillna(method='bfill')
output_data = output_data.fillna(method='ffill').fillna(method='bfill')

for i in range(len(most_rain_recorded_stations)):
    input_data_next = input_data[input_data[input_data.columns[0]] == most_rain_recorded_stations[i]]
    output_data_next = output_data[output_data[output_data.columns[0]] == most_rain_recorded_stations[i]]
    pecnet_preds, lstm_preds, prophet_preds = run_multiple(input_data_next, output_data_next, pecnet_preds, lstm_preds, prophet_preds, most_rain_recorded_stations[i])
    #pecnet_preds = run_pecnet(input_data_next, output_data_next, pecnet_preds, most_rain_recorded_stations[i])


In [ ]:
history1, history2, history3, history4, history5, history6, final_history = pecnet_preds['45.02'][2]

In [ ]:
fig, ((ax0,ax1),(ax2,ax3),(ax4,ax5)) = plt.subplots(3,2, figsize=(15,15))
ax0.plot(history1.history["loss"])
ax0.set_title("First Network")

ax1.plot(history2.history["loss"])
ax1.set_title("Second Network")

ax2.plot(history3.history["loss"])
ax2.set_title("Third Network")

ax3.plot(history4.history["loss"])
ax3.set_title("Fourth Network")

ax4.plot(history6.history["loss"])
ax4.set_title("Error Network")

ax5.plot(final_history.history["loss"])
ax5.set_title("Final Network")

In [ ]:
lstm_preds_list_updated = [[key] + value[3:] for key, value in lstm_preds.items()]
lstm_preds_list_updated = pd.DataFrame(lstm_preds_list_updated).rename({0: 'Station', 1: 'MAPE (with 0s)', 2: 'MSE (with 0s)', 3: 'MAE (with 0s)', 4: 'MAPE (without 0s)', 5: 'MSE (without 0s)', 6: 'MAE (without 0s)'}, axis='columns')

In [ ]:
print('PECNET MSE (with 0s)', np.mean(np.sqrt(lstm_preds_list_updated['MSE (with 0s)'])))
print('LSTM MSE (with 0s)', np.mean(np.sqrt(lstm_preds_list['MSE (with 0s)'])))
print()

print('PECNET MAE (with 0s)', np.mean(lstm_preds_list_updated['MAE (with 0s)']))
print('LSTM MAE (with 0s)', np.mean(lstm_preds_list['MAE (with 0s)']))
print()

print('PECNET MAPE (without 0s)', 100*np.mean(lstm_preds_list_updated['MAPE (without 0s)']), '%')
print('LSTM MAPE (without 0s)', 100*np.mean(lstm_preds_list['MAPE (without 0s)']), '%')
print()

print('PECNET MSE (without 0s)', np.mean(np.sqrt(lstm_preds_list_updated['MSE (without 0s)'])))
print('LSTM MSE (without 0s)', np.mean(np.sqrt(lstm_preds_list['MSE (without 0s)'])))
print()

print('PECNET MAE (without 0s)', np.mean(lstm_preds_list_updated['MAE (without 0s)']))
print('LSTM MAE (without 0s)', np.mean(lstm_preds_list['MAE (without 0s)']))


In [ ]:
pecnet_preds_list = [[key] + value[3:] for key, value in pecnet_preds.items()]
lstm_preds_list = [[key] + value[3:] for key, value in lstm_preds.items()]
prophet_preds_list = [[key] + value[3:] for key, value in prophet_preds.items()]

In [ ]:
pecnet_preds_list = [[key] + value for key, value in pecnet_preds.items()]
lstm_preds_list = [[key] + value for key, value in lstm_preds.items()]
prophet_preds_list = [[key] + value for key, value in prophet_preds.items()]

In [ ]:
pecnet_preds_list = pd.DataFrame(pecnet_preds_list)
lstm_preds_list = pd.DataFrame(lstm_preds_list)
prophet_preds_list = pd.DataFrame(prophet_preds_list)

In [ ]:
mask = pecnet_preds[list(pecnet_preds.keys())[0]][1] >= 1e-5
# Plot PECNET predictions vs true values
plt.figure(figsize=(10, 6))
plt.plot(pecnet_preds[list(pecnet_preds.keys())[0]][1][mask][-100:], label='True Values')
plt.plot(pecnet_preds[list(pecnet_preds.keys())[0]][0][mask][-100:], label='PECNET Predictions')
plt.plot(lstm_preds[list(lstm_preds.keys())[0]][0][mask][-100:], label='LSTM Predictions')
plt.plot(prophet_preds[list(prophet_preds.keys())[0]][0][mask][-100:], label='Prophet Predictions')
plt.title('PECNET Predictions vs True Values')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.show()

In [ ]:
pd.DataFrame(pecnet_preds_list).to_csv('pecnet_preds_raw.csv', index=False)
pd.DataFrame(lstm_preds_list).to_csv('lstm_preds_raw.csv', index=False)
pd.DataFrame(prophet_preds_list).to_csv('prophet_preds_raw.csv', index=False)

In [ ]:
pecnet_preds_list = pd.read_csv('pecnet_preds_raw.csv')
lstm_preds_list = pd.read_csv('lstm_preds_raw.csv')
prophet_preds_list = pd.read_csv('prophet_preds_raw.csv')

In [ ]:
true_values = []
for i in range(len(most_rain_recorded_stations)):
    #true_values[most_rain_recorded_stations[i]] = []
    input_data_next = input_data[input_data[input_data.columns[0]] == most_rain_recorded_stations[i]]
    output_data_next = output_data[output_data[output_data.columns[0]] == most_rain_recorded_stations[i]]
    _, _, _, _, _, _, _, _, _, _, _, output_test = preprocess_data(input_data_next, output_data_next)
    true_values.append(np.array(output_test))
true_values = np.concatenate(true_values)
mask_true = true_values > 1e-5

In [ ]:
np.std(true_values[mask_true])

In [ ]:
pecnet_preds_list = pd.DataFrame(pecnet_preds_list).rename({0: 'Station', 1: 'MAPE (with 0s)', 2: 'MSE (with 0s)', 3: 'MAE (with 0s)', 4: 'MAPE (without 0s)', 5: 'MSE (without 0s)', 6: 'MAE (without 0s)'}, axis='columns')
lstm_preds_list = pd.DataFrame(lstm_preds_list).rename({0: 'Station', 1: 'MAPE (with 0s)', 2: 'MSE (with 0s)', 3: 'MAE (with 0s)', 4: 'MAPE (without 0s)', 5: 'MSE (without 0s)', 6: 'MAE (without 0s)'}, axis='columns')
prophet_preds_list = pd.DataFrame(prophet_preds_list).rename({0: 'Station', 1: 'MAPE (with 0s)', 2: 'MSE (with 0s)', 3: 'MAE (with 0s)', 4: 'MAPE (without 0s)', 5: 'MSE (without 0s)', 6: 'MAE (without 0s)'}, axis='columns')

In [ ]:
print('PECNET MSE (with 0s)', np.mean(np.sqrt(pecnet_preds_list['MSE (with 0s)'])))
print('LSTM MSE (with 0s)', np.mean(np.sqrt(lstm_preds_list['MSE (with 0s)'])))
print('Prophet MSE (with 0s)', np.mean(np.sqrt(prophet_preds_list['MSE (with 0s)'])))
print()

print('PECNET MAE (with 0s)', np.mean(pecnet_preds_list['MAE (with 0s)']))
print('LSTM MAE (with 0s)', np.mean(lstm_preds_list['MAE (with 0s)']))
print('Prophet MAE (with 0s)', np.mean(prophet_preds_list['MAE (with 0s)']))
print()

print('PECNET MAPE (without 0s)', 100*np.mean(pecnet_preds_list['MAPE (without 0s)']), '%')
print('LSTM MAPE (without 0s)', 100*np.mean(lstm_preds_list['MAPE (without 0s)']), '%')
print('Prophet MAPE (without 0s)', 100*np.mean(prophet_preds_list['MAPE (without 0s)']), '%')
print()

print('PECNET MSE (without 0s)', np.mean(np.sqrt(pecnet_preds_list['MSE (without 0s)'])))
print('LSTM MSE (without 0s)', np.mean(np.sqrt(lstm_preds_list['MSE (without 0s)'])))
print('Prophet MSE (without 0s)', np.mean(np.sqrt(prophet_preds_list['MSE (without 0s)'])))
print()

print('PECNET MAE (without 0s)', np.mean(pecnet_preds_list['MAE (without 0s)']))
print('LSTM MAE (without 0s)', np.mean(lstm_preds_list['MAE (without 0s)']))
print('Prophet MAE (without 0s)', np.mean(prophet_preds_list['MAE (without 0s)']))
print()

print('Standard deviation of data points (with 0s)', np.std(true_values))
print('Standard deviation of data points (without 0s)', np.std(true_values[mask_true]))

In [ ]:
sum(lstm_preds['45.02'][1])

In [ ]:
sum(pecnet_preds['45.02'][1])

In [ ]:
sum(prophet_preds['45.02'][1])

In [ ]:
# Plot PECNET predictions vs true values
plt.figure(figsize=(10, 6))
plt.plot(output_test_filtered, label='True Values')
plt.plot(pecnet_predictions_filtered, label='PECNET Predictions')
plt.title('PECNET Predictions vs True Values')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.show()

In [ ]:
# Filter out target values less than 1e-5
mask1 = pecnet_preds[list(pecnet_preds.keys())[0]][1] >= 1e-5
output_test_filtered = pecnet_preds[list(pecnet_preds.keys())[0]][1][mask1]
pecnet_predictions_filtered = pecnet_preds[list(pecnet_preds.keys())[0]][0][mask1]
# Filter out target values less than 1e-5
mask2 = lstm_preds[list(pecnet_preds.keys())[0]][1] >= 1e-5
lstm_test_filtered = lstm_preds[list(pecnet_preds.keys())[0]][1][mask2]
lstm_predictions_filtered = lstm_preds[list(pecnet_preds.keys())[0]][0][mask2]
# Filter out target values less than 1e-5
mask3 = prophet_preds[list(pecnet_preds.keys())[0]][1] >= 1e-5
prophet_output_test_filtered = prophet_preds[list(pecnet_preds.keys())[0]][1][mask3]
prophet_predictions_filtered = prophet_preds[list(pecnet_preds.keys())[0]][0][mask3]

In [ ]:
pecnet_true_daily = []
pecnet_pred_daily = []
for i in range(0, len(pecnet_preds[list(pecnet_preds.keys())[0]][1])-6, 6):
    pecnet_true_daily.append(pecnet_preds[list(pecnet_preds.keys())[0]][1][i+5])
    pecnet_pred_daily.append(pecnet_preds[list(pecnet_preds.keys())[0]][0][i+5])
pecnet_true_daily = np.array(pecnet_true_daily)
pecnet_pred_daily = np.array(pecnet_pred_daily)

In [ ]:
def calculate_hourly_preds(pred_data, k):
    preds_weekly_mape = {}
    for stat in list(pred_data.keys()):
        preds_weekly_mape[stat] = []
        true_weekly = []
        pred_weekly = []
        for i in range(k, len(pred_data[stat][1])-6, 6):
            true_weekly.append(pred_data[stat][1][i])
            pred_weekly.append(pred_data[stat][0][i])
        true_weekly = np.array(true_weekly)
        pred_weekly = np.array(pred_weekly)
        mask4 = true_weekly >= 1e-5
        preds_weekly_mape[stat].append(100*mean_absolute_percentage_error(true_weekly[mask4], pred_weekly[mask4]))
    return preds_weekly_mape

def calculate_daily_preds(pred_data, k):
    preds_daily_mape = {}
    for stat in list(pred_data.keys()):
        preds_daily_mape[stat] = []
        true_daily = []
        pred_daily = []
        for i in range(k, len(pred_data[stat][1])-144, 144):
            true_daily_sum = 0
            pred_daily_sum = 0
            for j in range(24):
                true_daily_sum += pred_data[stat][1][i+6*j]
                pred_daily_sum += pred_data[stat][0][i+6*j]
            true_daily.append(true_daily_sum)
            pred_daily.append(pred_daily_sum)
        true_daily = np.array(true_daily)
        pred_daily = np.array(pred_daily)
        mask4 = true_daily >= 1e-5
        preds_daily_mape[stat].append(100*mean_absolute_percentage_error(true_daily[mask4], pred_daily[mask4]))
    return preds_daily_mape

def calculate_weekly_preds(pred_data, k):
    preds_weekly_mape = {}
    for stat in list(pred_data.keys()):
        preds_weekly_mape[stat] = []
        true_weekly = []
        pred_weekly = []
        for i in range(k, len(pred_data[stat][1])-1008, 1008):
            true_weekly_sum = 0
            pred_weekly_sum = 0
            for j in range(168):
                true_weekly_sum += pred_data[stat][1][i+6*j]
                pred_weekly_sum += pred_data[stat][0][i+6*j]
            true_weekly.append(true_weekly_sum)
            pred_weekly.append(pred_weekly_sum)
        true_weekly = np.array(true_weekly)
        pred_weekly = np.array(pred_weekly)
        mask4 = true_weekly >= 1e-5
        preds_weekly_mape[stat].append(100*mean_absolute_percentage_error(true_weekly[mask4], pred_weekly[mask4]))
    return preds_weekly_mape

In [ ]:
pecnet_preds_hourly_mape_list_means = []
lstm_preds_hourly_mape_list_means = []
prophet_preds_hourly_mape_list_means = []
pecnet_preds_daily_mape_list_means = []
lstm_preds_daily_mape_list_means = []
prophet_preds_daily_mape_list_means = []
pecnet_preds_weekly_mape_list_means = []
lstm_preds_weekly_mape_list_means = []
prophet_preds_weekly_mape_list_means = []

for k in range(6):
    pecnet_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(pecnet_preds, k).items()])
    lstm_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(lstm_preds, k).items()])
    prophet_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(prophet_preds, k).items()])

    pecnet_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(pecnet_preds, k).items()])
    lstm_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(lstm_preds, k).items()])
    prophet_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(prophet_preds, k).items()])

    pecnet_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(pecnet_preds, k).items()])
    lstm_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(lstm_preds, k).items()])
    prophet_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(prophet_preds, k).items()])

    pecnet_preds_hourly_mape_list_means.append(np.mean(pecnet_preds_hourly_mape_list[1]))
    lstm_preds_hourly_mape_list_means.append(np.mean(lstm_preds_hourly_mape_list[1]))
    prophet_preds_hourly_mape_list_means.append(np.mean(prophet_preds_hourly_mape_list[1]))
    pecnet_preds_daily_mape_list_means.append(np.mean(pecnet_preds_daily_mape_list[1]))
    lstm_preds_daily_mape_list_means.append(np.mean(lstm_preds_daily_mape_list[1]))
    prophet_preds_daily_mape_list_means.append(np.mean(prophet_preds_daily_mape_list[1]))
    pecnet_preds_weekly_mape_list_means.append(np.mean(pecnet_preds_weekly_mape_list[1]))
    lstm_preds_weekly_mape_list_means.append(np.mean(lstm_preds_weekly_mape_list[1]))
    prophet_preds_weekly_mape_list_means.append(np.mean(prophet_preds_weekly_mape_list[1]))

In [ ]:
print('PECNET HOURLY MAPE (without 0s)', np.mean(pecnet_preds_hourly_mape_list_means), '%')
print('LSTM HOURLY MAPE (without 0s)', np.mean(lstm_preds_hourly_mape_list_means), '%')
print('Prophet HOURLY MAPE (without 0s)', np.mean(prophet_preds_hourly_mape_list_means), '%')
print()
print('PECNET DAILY MAPE (without 0s)', np.mean(pecnet_preds_daily_mape_list_means), '%')
print('LSTM DAILY MAPE (without 0s)', np.mean(lstm_preds_daily_mape_list_means), '%')
print('PROPHET DAILY MAPE (without 0s)', np.mean(prophet_preds_daily_mape_list_means), '%')
print()
print('PECNET WEEKLY MAPE (without 0s)', np.mean(pecnet_preds_weekly_mape_list_means), '%')
print('LSTM WEEKLY MAPE (without 0s)', np.mean(lstm_preds_weekly_mape_list_means), '%')
print('PROPHET WEEKLY MAPE (without 0s)', np.mean(prophet_preds_weekly_mape_list_means), '%')

In [ ]:
k=0
pecnet_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(pecnet_preds, k).items()])
lstm_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(lstm_preds, k).items()])
prophet_preds_hourly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_hourly_preds(prophet_preds, k).items()])

pecnet_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(pecnet_preds, k).items()])
lstm_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(lstm_preds, k).items()])
prophet_preds_daily_mape_list = pd.DataFrame([[key] + value for key, value in calculate_daily_preds(prophet_preds, k).items()])

pecnet_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(pecnet_preds, k).items()])
lstm_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(lstm_preds, k).items()])
prophet_preds_weekly_mape_list = pd.DataFrame([[key] + value for key, value in calculate_weekly_preds(prophet_preds, k).items()])

print('PECNET HOURLY MAPE (without 0s)', np.mean(pecnet_preds_hourly_mape_list[1]), '%')
print('LSTM HOURLY MAPE (without 0s)', np.mean(lstm_preds_hourly_mape_list[1]), '%')
print('Prophet HOURLY MAPE (without 0s)', np.mean(prophet_preds_hourly_mape_list[1]), '%')
print()
print('PECNET DAILY MAPE (without 0s)', np.mean(pecnet_preds_daily_mape_list[1]), '%')
print('LSTM DAILY MAPE (without 0s)', np.mean(lstm_preds_daily_mape_list[1]), '%')
print('PROPHET DAILY MAPE (without 0s)', np.mean(prophet_preds_daily_mape_list[1]), '%')
print()
print('PECNET WEEKLY MAPE (without 0s)', np.mean(pecnet_preds_weekly_mape_list[1]), '%')
print('LSTM WEEKLY MAPE (without 0s)', np.mean(lstm_preds_weekly_mape_list[1]), '%')
print('PROPHET WEEKLY MAPE (without 0s)', np.mean(prophet_preds_weekly_mape_list[1]), '%')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

station_count = np.arange(1, len(pecnet_preds_daily_mape_list) + 1)

# Combine the data into a single DataFrame for plotting
data = {
    'Station': station_count,
    'PECNET_MAPE_daily': pecnet_preds_daily_mape_list[1],
    'LSTM_MAPE_daily': lstm_preds_daily_mape_list[1],
    'Prophet_MAPE_daily': prophet_preds_daily_mape_list[1],
    'PECNET_MAPE_weekly': pecnet_preds_weekly_mape_list[1],
    'LSTM_MAPE_weekly': lstm_preds_weekly_mape_list[1],
    'Prophet_MAPE_weekly': prophet_preds_weekly_mape_list[1],
}
df = pd.DataFrame(data)

# Function to create boxplots with error bars for a specific metric
def plot_boxplot_with_error_bars(ax, metric_name, metric_cols):
    data_to_plot = [df[col] for col in metric_cols]
    boxprops = dict(facecolor='lightblue', color='blue')
    medianprops = dict(color='red', linewidth=2)
    meanprops = dict(marker='o', markerfacecolor='green', markeredgecolor='None', markersize=7)
    flierprops = dict(marker='o', markerfacecolor='none', markeredgecolor='purple', markersize=7)

    ax.boxplot(data_to_plot, labels=['PECNET', 'LSTM', 'Prophet'], boxprops=boxprops, medianprops=medianprops, meanprops=meanprops, flierprops=flierprops, patch_artist=True, showmeans=True, showfliers=False)
    ax.set_title(metric_name.split(' ')[0], fontsize=20)
    ax.set_ylabel(metric_name, fontsize=18)
    ax.grid(True)
    ax.tick_params(axis='both', which='major', labelsize=16)

    # Creating custom legend
    handles = [
        mpatches.Patch(color='lightblue', label='25%-75%'),
        mlines.Line2D([], [], color='red', label='Median'),
        mlines.Line2D([], [], color='green', marker='o', linestyle='None', markersize=7, label='Mean'),
        #mlines.Line2D([], [], color='purple', marker='o', markerfacecolor='none', linestyle='None', markersize=7, label='Outliers')
    ]

    ax.legend(handles=handles, loc='upper left', fontsize=10)

# Plotting without zeros (two graphs above, one below centered horizontally)
fig = plt.figure(figsize=(16, 12))

# First plot
ax1 = fig.add_axes([0.1, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax1, 'DAILY MAPE (%)', ['PECNET_MAPE_daily', 'LSTM_MAPE_daily', 'Prophet_MAPE_daily'])

# Second plot
ax2 = fig.add_axes([0.55, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax2, 'WEEKLY MAPE (%)', ['PECNET_MAPE_weekly', 'LSTM_MAPE_weekly', 'Prophet_MAPE_weekly'])

plt.tight_layout()
plt.show()


In [ ]:
mask4 = pecnet_true_daily >= 1e-5
pecnet_true_daily_filtered = pecnet_true_daily[mask4]
pecnet_pred_daily_filtered = pecnet_pred_daily[mask4]

In [ ]:
mean_absolute_percentage_error(pecnet_true_daily_filtered, pecnet_pred_daily_filtered)

In [ ]:
# Plot PECNET predictions vs true values
plt.figure(figsize=(10, 6))
plt.plot(pecnet_true_daily_filtered, label='True Values')
plt.plot(pecnet_pred_daily_filtered, label='PECNET Predictions')
plt.title('PECNET Predictions vs True Values')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.show()

In [ ]:
len(pecnet_pred_daily)

In [ ]:
len(pecnet_preds[list(pecnet_preds.keys())[0]][1])

In [ ]:
np.mean(pecnet_predictions_filtered)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Example data, replace with actual data
pecnet_preds = pecnet_preds_list.copy().sort_values(by='Station')
lstm_preds = lstm_preds_list.copy().sort_values(by='Station')
prophet_preds = prophet_preds_list.copy().sort_values(by='Station')
station_count = np.arange(1, len(pecnet_preds) + 1)
pecnet_preds['MSE (without 0s)'] = np.sqrt(pecnet_preds['MSE (without 0s)'])
lstm_preds['MSE (without 0s)'] = np.sqrt(lstm_preds['MSE (without 0s)'])
prophet_preds['MSE (without 0s)'] = np.sqrt(prophet_preds['MSE (without 0s)'])
pecnet_preds['MSE (with 0s)'] = np.sqrt(pecnet_preds['MSE (with 0s)'])
lstm_preds['MSE (with 0s)'] = np.sqrt(lstm_preds['MSE (with 0s)'])
prophet_preds['MSE (with 0s)'] = np.sqrt(prophet_preds['MSE (with 0s)'])
pecnet_preds['MAPE (without 0s)'] *= 100
lstm_preds['MAPE (without 0s)'] *= 100
prophet_preds['MAPE (without 0s)'] *= 100

# Function to create bar charts for a specific metric without zeros
def plot_metric_without_zeros(ax, metric_name, metric_col):
    width = 0.25
    ax.bar(station_count - width, pecnet_preds[metric_col], width=width, label='PECNET')
    ax.bar(station_count, lstm_preds[metric_col], width=width, label='LSTM')
    ax.bar(station_count + width, prophet_preds[metric_col], width=width, label='Prophet')
    title = ''
    if metric_name == 'MAPE (%)':
        title = 'MAPE'
    elif metric_name == 'RMSE (mm/h)':
        title = 'RMSE'
    else:
        title = 'MAE'
    ax.set_title(title, fontsize=20)
    ax.set_xlabel('Station', fontsize=18)
    ax.set_ylabel(metric_name, fontsize=18)
    ax.grid(True)
    ax.legend(fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=16)

# Function to create bar charts for a specific metric with zeros
def plot_metric_with_zeros(ax, metric_name, metric_col):
    width = 0.25
    ax.bar(station_count - width, pecnet_preds[metric_col], width=width, label='PECNET')
    ax.bar(station_count, lstm_preds[metric_col], width=width, label='LSTM')
    ax.bar(station_count + width, prophet_preds[metric_col], width=width, label='Prophet')
    title = ''
    if metric_name == 'MAPE (%)':
        title = 'MAPE'
    elif metric_name == 'RMSE (mm/h)':
        title = 'RMSE'
    else:
        title = 'MAE'
    ax.set_title(title, fontsize=20)
    ax.set_xlabel('Station', fontsize=18)
    ax.set_ylabel(metric_name, fontsize=18)
    ax.grid(True)
    ax.legend(fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=16)

# Plotting without zeros (two graphs above, one below centered horizontally)
fig = plt.figure(figsize=(16, 12))

# First row, first column
ax1 = fig.add_axes([0.1, 0.55, 0.4, 0.4])
plot_metric_without_zeros(ax1, 'MAPE (%)', 'MAPE (without 0s)')

# First row, second column
ax2 = fig.add_axes([0.55, 0.55, 0.4, 0.4])
plot_metric_without_zeros(ax2, 'RMSE (mm/h)', 'MSE (without 0s)')

# Second row, centered
ax3 = fig.add_axes([0.325, 0.05, 0.4, 0.4])
plot_metric_without_zeros(ax3, 'MAE (mm/h)', 'MAE (without 0s)')

plt.tight_layout()
plt.show()

# Plotting with zeros (excluding MAPE with zeros)
fig = plt.figure(figsize=(16, 12))

# First plot
ax4 = fig.add_axes([0.1, 0.55, 0.4, 0.4])
plot_metric_with_zeros(ax4, 'RMSE (mm/h)', 'MSE (with 0s)')

# Second plot
ax5 = fig.add_axes([0.55, 0.55, 0.4, 0.4])
plot_metric_with_zeros(ax5, 'MAE (mm/h)', 'MAE (with 0s)')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Example data, replace with actual data
pecnet_preds = pecnet_preds_list.copy().sort_values(by='Station')
lstm_preds = lstm_preds_list.copy().sort_values(by='Station')
prophet_preds = prophet_preds_list.copy().sort_values(by='Station')
station_count = np.arange(1, len(pecnet_preds) + 1)
pecnet_preds['MSE (without 0s)'] = np.sqrt(pecnet_preds['MSE (without 0s)'])
lstm_preds['MSE (without 0s)'] = np.sqrt(lstm_preds['MSE (without 0s)'])
prophet_preds['MSE (without 0s)'] = np.sqrt(prophet_preds['MSE (without 0s)'])
pecnet_preds['MSE (with 0s)'] = np.sqrt(pecnet_preds['MSE (with 0s)'])
lstm_preds['MSE (with 0s)'] = np.sqrt(lstm_preds['MSE (with 0s)'])
prophet_preds['MSE (with 0s)'] = np.sqrt(prophet_preds['MSE (with 0s)'])
pecnet_preds['MAPE (without 0s)'] *= 100
lstm_preds['MAPE (without 0s)'] *= 100
prophet_preds['MAPE (without 0s)'] *= 100

# Combine the data into a single DataFrame for plotting
data = {
    'Station': station_count,
    'PECNET_MSE_wo0s': pecnet_preds['MSE (without 0s)'],
    'LSTM_MSE_wo0s': lstm_preds['MSE (without 0s)'],
    'Prophet_MSE_wo0s': prophet_preds['MSE (without 0s)'],
    'PECNET_MSE_w0s': pecnet_preds['MSE (with 0s)'],
    'LSTM_MSE_w0s': lstm_preds['MSE (with 0s)'],
    'Prophet_MSE_w0s': prophet_preds['MSE (with 0s)'],
    'PECNET_MAPE_wo0s': pecnet_preds['MAPE (without 0s)'],
    'LSTM_MAPE_wo0s': lstm_preds['MAPE (without 0s)'],
    'Prophet_MAPE_wo0s': prophet_preds['MAPE (without 0s)'],
    'PECNET_MAE_wo0s': pecnet_preds['MAE (without 0s)'],
    'LSTM_MAE_wo0s': lstm_preds['MAE (without 0s)'],
    'Prophet_MAE_wo0s': prophet_preds['MAE (without 0s)'],
    'PECNET_MAE_w0s': pecnet_preds['MAE (with 0s)'],
    'LSTM_MAE_w0s': lstm_preds['MAE (with 0s)'],
    'Prophet_MAE_w0s': prophet_preds['MAE (with 0s)'],
    'PECNET_MAPE_weekly': pecnet_preds_weekly_mape_list[1],
    'LSTM_MAPE_weekly': lstm_preds_weekly_mape_list[1],
    'Prophet_MAPE_weekly': prophet_preds_weekly_mape_list[1],
}
df = pd.DataFrame(data)

# Function to create boxplots with error bars for a specific metric
def plot_boxplot_with_error_bars(ax, metric_name, metric_cols):
    data_to_plot = [df[col] for col in metric_cols]
    boxprops = dict(facecolor='lightblue', color='blue')
    medianprops = dict(color='red', linewidth=2)
    meanprops = dict(marker='o', markerfacecolor='green', markeredgecolor='None', markersize=7)
    flierprops = dict(marker='o', markerfacecolor='none', markeredgecolor='purple', markersize=7)

    ax.boxplot(data_to_plot, labels=['PECNET', 'LSTM', 'Prophet'], boxprops=boxprops, medianprops=medianprops, meanprops=meanprops, flierprops=flierprops, patch_artist=True, showmeans=True)
    ax.set_title(metric_name.split('(')[0], fontsize=20)
    ax.set_ylabel(metric_name, fontsize=18)
    if metric_name == 'WEEKLY MAPE (%)':
        ax.set_ylabel('MAPE (%)', fontsize=18)
    ax.grid(True)
    ax.tick_params(axis='both', which='major', labelsize=16)

    # Creating custom legend
    handles = [
        mpatches.Patch(color='lightblue', label='25%-75%'),
        mlines.Line2D([], [], color='red', label='Median'),
        mlines.Line2D([], [], color='green', marker='o', linestyle='None', markersize=7, label='Mean'),
        mlines.Line2D([], [], color='purple', marker='o', markerfacecolor='none', linestyle='None', markersize=7, label='Outliers')
    ]

    ax.legend(handles=handles, loc='upper left', fontsize=10)

# Plotting without zeros (two graphs above, one below centered horizontally)
fig = plt.figure(figsize=(16, 12))

# First row, first column
ax1 = fig.add_axes([0.05, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax1, 'MAPE (%)', ['PECNET_MAPE_wo0s', 'LSTM_MAPE_wo0s', 'Prophet_MAPE_wo0s'])

# First row, second column
ax2 = fig.add_axes([0.55, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax2, 'WEEKLY MAPE (%)', ['PECNET_MAPE_weekly', 'LSTM_MAPE_weekly', 'Prophet_MAPE_weekly'])

# Second row, first column
ax3 = fig.add_axes([0.05, 0.05, 0.4, 0.4])
plot_boxplot_with_error_bars(ax3, 'RMSE (mm/h)', ['PECNET_MSE_wo0s', 'LSTM_MSE_wo0s', 'Prophet_MSE_wo0s'])

# Second row, second column
ax4 = fig.add_axes([0.55, 0.05, 0.4, 0.4])
plot_boxplot_with_error_bars(ax4, 'MAE (mm/h)', ['PECNET_MAE_wo0s', 'LSTM_MAE_wo0s', 'Prophet_MAE_wo0s'])

plt.tight_layout()
plt.show()

# Plotting with zeros (excluding MAPE with zeros)
fig = plt.figure(figsize=(16, 12))

# First plot
ax5 = fig.add_axes([0.05, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax5, 'RMSE (mm/h)', ['PECNET_MSE_w0s', 'LSTM_MSE_w0s', 'Prophet_MSE_w0s'])

# Second plot
ax6 = fig.add_axes([0.55, 0.55, 0.4, 0.4])
plot_boxplot_with_error_bars(ax6, 'MAE (mm/h)', ['PECNET_MAE_w0s', 'LSTM_MAE_w0s', 'Prophet_MAE_w0s'])

plt.tight_layout()
plt.show()


In [ ]:
most_rain_recorded_stations[2] = '26.10'

In [ ]:
most_rain_recorded_stations

In [ ]:
station_coordinates

In [ ]:
station_coordinates = []
for station in most_rain_recorded_stations:
    station_id = str(station).split('.')
    if len(station_id[0]) == 1:
        station_id[0] = '0'+ station_id[0]
    if len(station_id[1]) == 1:
        station_id[1] = '0'+ station_id[1]
    station_id = station_id[0] + '.' + station_id[1]
    raw_station_coordinate = pd.read_csv(os.path.join('StationDataRaw', station_id + '_raw.csv'), usecols=['enlem', 'boylam'], dtype=str)
    enlem, boylam = raw_station_coordinate['enlem'][0], raw_station_coordinate['boylam'][0]
    station_coordinates.append([enlem, boylam])

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Read the data from the JSON file
file_path = 'tr-cities.json'  # Replace with the actual file path
data = gpd.read_file(file_path)

# Plot the map
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='lightgrey', edgecolor='black')

for enlem, boylam in station_coordinates:
    # Mark the specified location on the map
    target_location = gpd.GeoDataFrame({'geometry': gpd.points_from_xy([enlem], [boylam])})
    target_location.plot(ax=ax, color='red', markersize=80)
ax.axis('off')
ax.legend()
# Show the plot
plt.legend()
plt.show()


In [ ]:
import folium
import pandas as pd

# Create a map centered around Turkey
map_turkey = folium.Map(location=[39.9208, 32.8541], zoom_start=6, tiles='cartodbpositron')

# Iterate over each row in the data and add a circle marker for each station
for enlem, boylam in station_coordinates:
    enlem = float(enlem.replace(',','.'))
    boylam = float(boylam.replace(',','.'))
    col = 'blue'
    
    # Add a circle marker to the map
    folium.CircleMarker(location=[enlem, boylam], radius=3, fill=True, color=col, fill_color='blue').add_to(map_turkey)

# Save the map as an HTML file
map_turkey.save('turkey_map.html')


In [ ]:
import folium
import pandas as pd

# Read the CSV file
data = pd.read_csv('station_list.csv')

# Create a map centered around Turkey
all_stations_marked = folium.Map(location=[39.9208, 32.8541], zoom_start=6, tiles='cartodbpositron')

# Iterate over each row in the data and add a circle marker for each station
for index, row in data.iterrows():
    code = row['code']
    name = row['name']
    lat = row['lat']
    lon = row['lon']
    col = 'blue'
    if code == '63.13':
        col = 'blue'
    elif code == '63.11':
        col = 'green'
    elif code == '63.12':
        col = 'red'
    else:
        continue
    
    # Add a circle marker to the map
    folium.CircleMarker(location=[lat, lon], radius=3, tooltip=f"{code}: {name}", fill=True, color=col, fill_color='blue').add_to(all_stations_marked)

# Save the map as an HTML file
all_stations_marked.save('all_stations_marked.html')

In [ ]:
input_take = input_data[input_data['Station'] == 45.02]
output_take = output_data[output_data['Station'] == 45.02]

In [ ]:
input_take = input_take.drop(['Station', 'Date'], axis=1)
output_take = output_take.drop(['Station', 'Date'], axis=1)

In [ ]:
import ast

input_take = input_take.applymap(lambda x: ast.literal_eval(x)[3] if isinstance(x, str) and x.startswith('[') else x)

In [ ]:
input_take['Target'] = list(output_take['ARF'])

In [ ]:
correlation_matrix = input_take.corr(method='pearson')

In [ ]:
import seaborn as sns
# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()